In [0]:
# Common imports
import pandas as pd
from IPython.display import Markdown, display, clear_output

In [0]:
import _pickle as cPickle
from pathlib import Path

def dumpPickle(fileName, content):
    pickleFile = open(fileName, 'wb')
    cPickle.dump(content, pickleFile, -1)
    pickleFile.close()

def loadPickle(fileName):    
    file = open(fileName, 'rb')
    content = cPickle.load(file)
    file.close()
    
    return content
    
def pickleExists(fileName):
    file = Path(fileName)
    
    if file.is_file():
        return True
    
    return False

extract sarcasm from the sentences# New Section

In [0]:
#tweet
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import csv
import time

# you need to add yours key and tokens here...............................
ckey = ''
csecret = ''
atoken = ''
asecret = ''


class listener(StreamListener):
    def on_data(self, data):
        try:
            tweettext = data.split(',"text":"')[1].split('","source')[0]
            if tweettext[
               0:2] != 'RT':  # and '#sarcasm' not in tweettext and len(tweettext.split())>5:  #Basic additionnal filters if necessary.
                print(tweettext)
                saveFile = open('twitDB_sarcasm.csv', 'a')
                saveFile.write(tweettext)
                saveFile.write('\n')
                saveFile.close()
            return True
        except BaseException as e:
            print('Failed ondata,', str(e))
            time.sleep(5)

    def on_error(self, status):
        print(status)


auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

# NY = [74,40,-73,41]
twitterStream = Stream(auth, listener())
twitterStream.filter(track=["#sarcasm"])
# twitterStream.filter(locations=[-122.75,36.8,-121.75,37.8,-74,40,-73,41])#locations=[-6.38,49.87,1.77,55.81])

In [0]:
#load sentences
import csv, collections, os
import numpy as np
import nltk


class load_sent_word_net(object):
    def __init__(self):
        sent_scores = collections.defaultdict(list)

        with open(os.path.join(os.path.dirname(os.path.realpath(__file__)), 'SentiWordNet_3.0.0_20130122.txt'),
                  'r') as csvfile:

            reader = csv.reader(csvfile, delimiter='\t', quotechar='"')

            for line in reader:
                if line[0].startswith('#'):
                    continue
                if len(line) == 1:
                    continue

                POS, ID, PosScore, NegScore, SynsetTerms, Gloss = line

                if len(POS) == 0 or len(ID) == 0:
                    continue

                for term in SynsetTerms.split(" "):
                    term = term.split("#")[0]
                    term = term.replace("-", " ").replace("_", " ")
                    key = "%s/%s" % (POS, term.split("#")[0])
                    sent_scores[key].append((float(PosScore), float(NegScore)))
                    # this is where magic happens. You classify all words by + or -.

        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value, axis=0)

        self.sent_scores = sent_scores

    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)

    def score_sentence(self, sentence):
        pos = nltk.pos_tag(sentence)
        mean_score = np.array([0.0, 0.0])
        for j in range(len(pos)):
            mean_score += self.score(pos[j][0], pos[j][1])

        return mean_score

    def score(self, word, pos):
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] == 'VB':
            pos_type = 'v'
        elif pos[0:2] == 'RB':  # adverb
            pos_type = 'r'
        else:
            pos_type = 0

        if pos_type != 0:
            dic_loc = pos_type + '/' + word
            pos_neg_scores = self.sent_scores[dic_loc]  # dictionary location
            if len(pos_neg_scores) == 2:
                return pos_neg_scores
            else:
                return np.array([0.0, 0.0])
        else:
            return np.array([0.0, 0.0])

    def posvector(self, sentence):  # position vector

        pos_vector = nltk.pos_tag(sentence)
        vector = np.zeros(4)

        for j in range(len(sentence)):
            pos = pos_vector[j][1]
            if pos[0:2] == 'NN':
                vector[0] += 1
            elif pos[0:2] == 'JJ':
                vector[1] += 1
            elif pos[0:2] == 'VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1

        return vector

In [0]:
#topic
from gensim import corpora, models, similarities
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

class topic(object):
    def __init__(self, nbtopic=100, alpha=1, model=None, dicttp=None):
        self.nbtopic = nbtopic
        self.porter = nltk.PorterStemmer()
        self.alpha = alpha
        self.stop = stopwords.words('english') + ['.', '!', '?', '"', '...', '\\', "''", '[', ']', '~', "'m", "'s", ';',
                                                  ':', '..', '$']
        if model != None and dicttp != None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary = corpora.Dictionary.load(dicttp)

    def fit(self, documents):
        documents_mod = [replace_reg(sentence).encode() for sentence in documents]
        tokens = [nltk.word_tokenize(sentence).encode() for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]

        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus, id2word=self.dictionary, num_topics=self.nbtopic, alpha=self.alpha)

        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')

    def get_topic(self, topic_number):
        return self.lda.print_topic(topic_number)

    def transform(self, sentence):
        sentence_mod = exp_replace.replace_reg(sentence).encode()
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop]
        corpus_sentence = self.dictionary.doc2bow(tokens)

        return self.lda[corpus_sentence]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#exp replace
import nltk
import re

emo_repl = {
    # good emotions
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " ood ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",

    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    # bad emotions
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad "
}

emo_repl2 = {
    # good emotions
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",

    # bad emotions
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad "
}

# general
re_repl = {
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not"
}

emo_repl_order = [k for (k_len, k) in reversed(sorted([(len(k), k) for k in list(emo_repl.keys())]))]
emo_repl_order2 = [k for (k_len, k) in reversed(sorted([(len(k), k) for k in list(emo_repl2.keys())]))]


def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k, emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r, repl, sentence2)
    return sentence2
    print (sentence2)


def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k, emo_repl2[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r, repl, sentence2)
    return sentence2
    print(sentence2)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# this is how we extract features from data. We make a dictionary for all features and add features into it.
import nltk
import numpy as np
import string
from textblob import TextBlob
porter = nltk.PorterStemmer()
sentiments = load_sent_word_net()

# all features calling.....
def dialogue_act_features(sentence, topic_modeler):
    features = {}

    grams_feature(features, sentence)
    sent_feature(features, sentence)
    pos_feature(features, sentence)
    cap_feature(features, sentence)
    topic_feature(features, sentence, topic_modeler)

    return features


def grams_feature(features, sentence):
    sentence_reg = exp_replace.replace_reg(sentence)

    tokens = nltk.word_tokenize(sentence_reg)
    tokens = [porter.stem(t.lower()) for t in tokens]
    bigrams = nltk.bigrams(tokens)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = tokens + bigrams

    for t in grams:
        features['contains(%s)' % t] = 1.0


def sent_feature(features, sentence):
    sentence_sentiment = exp_replace.replace_emo(sentence)
    tokens = nltk.word_tokenize(sentence_sentiment)
    tokens = [(t.lower()) for t in tokens]

    mean_sentiment = sentiments.score_sentence(tokens)
    features['Positive sentiment'] = mean_sentiment[0]
    features['Negative sentiment'] = mean_sentiment[1]
    features['Sentiment'] = mean_sentiment[0] - mean_sentiment[1]
    try:
        blob = TextBlob(
            "".join([" " + i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip())
        features['Blob sentiment'] = blob.sentiment.polarity
        features['Blob subjectivity'] = blob.sentiment.subjectivity
    except:
        features['Blob sentiment'] = 0.0
        features['Blob subjectivity'] = 0.0

    # Split in 2
    if len(tokens) == 1:
        tokens += ['.']
    f = tokens[0:len(tokens) / 2]
    s = tokens[len(tokens) / 2:]

    mean_sentiment_f = sentiments.score_sentence(f)
    features['Positive sentiment 1/2'] = mean_sentiment_f[0]
    features['Negative sentiment 1/2'] = mean_sentiment_f[1]
    features['Sentiment 1/2'] = mean_sentiment_f[0] - mean_sentiment_f[1]

    mean_sentiment_s = sentiments.score_sentence(s)
    features['Positive sentiment 2/2'] = mean_sentiment_s[0]
    features['Negative sentiment 2/2'] = mean_sentiment_s[1]
    features['Sentiment 2/2'] = mean_sentiment_s[0] - mean_sentiment_s[1]

    features['Sentiment contrast 2'] = np.abs(features['Sentiment 1/2'] - features['Sentiment 2/2'])

    # TextBlob sentiment analysis
    try:
        blob = TextBlob(
            "".join([" " + i if not i.startswith("'") and i not in string.punctuation else i for i in f]).strip())
        features['Blob sentiment 1/2'] = blob.sentiment.polarity
        features['Blob subjectivity 1/2'] = blob.sentiment.subjectivity
    except:
        features['Blob sentiment 1/2'] = 0.0
        features['Blob subjectivity 1/2'] = 0.0
    try:
        blob = TextBlob(
            "".join([" " + i if not i.startswith("'") and i not in string.punctuation else i for i in s]).strip())
        features['Blob sentiment 2/2'] = blob.sentiment.polarity
        features['Blob subjectivity 2/2'] = blob.sentiment.subjectivity
    except:
        features['Blob sentiment 2/2'] = 0.0
        features['Blob subjectivity 2/2'] = 0.0

    features['Blob Sentiment contrast 2'] = np.abs(features['Blob sentiment 1/2'] - features['Blob sentiment 2/2'])

    # Split in 3
    if len(tokens) == 2:
        tokens += ['.']
    f = tokens[0:len(tokens) / 3]
    s = tokens[len(tokens) / 3:2 * len(tokens) / 3]
    t = tokens[2 * len(tokens) / 3:]

    mean_sentiment_f = sentiments.score_sentence(f)
    features['Positive sentiment 1/3'] = mean_sentiment_f[0]
    features['Negative sentiment 1/3'] = mean_sentiment_f[1]
    features['Sentiment 1/3'] = mean_sentiment_f[0] - mean_sentiment_f[1]

    mean_sentiment_s = sentiments.score_sentence(s)
    features['Positive sentiment 2/3'] = mean_sentiment_s[0]
    features['Negative sentiment 2/3'] = mean_sentiment_s[1]
    features['Sentiment 2/3'] = mean_sentiment_s[0] - mean_sentiment_s[1]

    mean_sentiment_t = sentiments.score_sentence(t)
    features['Positive sentiment 3/3'] = mean_sentiment_t[0]
    features['Negative sentiment 3/3'] = mean_sentiment_t[1]
    features['Sentiment 3/3'] = mean_sentiment_t[0] - mean_sentiment_t[1]

    features['Sentiment contrast 3'] = np.abs(features['Sentiment 1/3'] - features['Sentiment 3/3'])

    try:
        blob = TextBlob(
            "".join([" " + i if not i.startswith("'") and i not in string.punctuation else i for i in f]).strip())
        features['Blob sentiment 1/3'] = blob.sentiment.polarity
        features['Blob subjectivity 1/3'] = blob.sentiment.subjectivity
    except:
        features['Blob sentiment 1/3'] = 0.0
        features['Blob subjectivity 1/3'] = 0.0
    try:
        blob = TextBlob(
            "".join([" " + i if not i.startswith("'") and i not in string.punctuation else i for i in s]).strip())
        features['Blob sentiment 2/3'] = blob.sentiment.polarity
        features['Blob subjectivity 2/3'] = blob.sentiment.subjectivity
    except:
        features['Blob sentiment 2/3'] = 0.0
        features['Blob subjectivity 2/3'] = 0.0
    try:
        blob = TextBlob(
            "".join([" " + i if not i.startswith("'") and i not in string.punctuation else i for i in t]).strip())
        features['Blob sentiment 3/3'] = blob.sentiment.polarity
        features['Blob subjectivity 3/3'] = blob.sentiment.subjectivity
    except:
        features['Blob sentiment 3/3'] = 0.0
        features['Blob subjectivity 3/3'] = 0.0

    features['Blob Sentiment contrast 3'] = np.abs(features['Blob sentiment 1/3'] - features['Blob sentiment 3/3'])


def pos_feature(features, sentence):
    sentence_pos = exp_replace.replace_emo(sentence)
    tokens = nltk.word_tokenize(sentence_pos)
    tokens = [(t.lower()) for t in tokens]
    pos_vector = sentiments.posvector(tokens)
    for j in range(len(pos_vector)):
        features['POS' + str(j + 1)] = pos_vector[j]


def cap_feature(features, sentence):
    counter = 0
    treshold = 4
    for j in range(len(sentence)):
        counter += int(sentence[j].isupper())
    features['Capitalization'] = int(counter >= treshold)  # true : 1, false : 0


def topic_feature(features, sentence, topic_modeler):
    topics = topic_modeler.transform(sentence)

    for j in range(len(topics)):
        features['Topic :' + str(topics[j][0])] = topics[j][1]

In [0]:
#evaluate
import numpy as np
import pickle
import os


# pickle files....
obj1 = open(os.path.join(os.path.dirname(os.path.realpath(__file__)), 'vecdict.p'), 'r')
obj2 = open(os.path.join(os.path.dirname(os.path.realpath(__file__)), 'classif.p'), 'r')

# vector of features of all the data.......
vec = pickle.load(obj1)
# SVC classifier that is already trained in traintest.py file......
classifier = pickle.load(obj2)

obj1.close()
obj2.close()

# this is required because of various OS have various slashes in paths...
topic_mod = topic(model=os.path.join(os.path.dirname(os.path.realpath(__file__)), 'topics.tp'),
                        dicttp=os.path.join(os.path.dirname(os.path.realpath(__file__)), 'topics_dict.tp'))


def tweetscore(sentence):
    features = feature_extract.dialogue_act_features(sentence, topic_mod)
    # classifier can only get data in numerical from so convert in vector form.
    features_vec = vec.transform(features)
    score = classifier.decision_function(features_vec)[0]
    # sigmoid and other manipulations........
    per = int(round(2.0 * (1.0 / (1.0 + np.exp(-score)) - 0.5) * 1000.0))

    return per

In [0]:
#sarcasm detector
import os
import flask, flask.views
from flask import Markup
from flask import jsonify

app = flask.Flask(__name__)


class Main(flask.views.MethodView):
    def get(self):
        return flask.render_template('index.html')



app.add_url_rule('/', view_func=Main.as_view('main'), methods=["GET"])



@app.route('/_compute')
def compute():
    sentence = flask.request.args.get('sentence')
    percentage = evaluate.tweetscore(str(sentence))
    return jsonify(result=percentage)

In [0]:
#preprocess
import numpy as np
import csv
import re


def preprocessing(csv_file_object):
    data = []
    length = []
    remove_hashtags = re.compile(r'#\w+\s?')
    remove_friendtag = re.compile(r'@\w+\s?')
    remove_sarcasm = re.compile(re.escape('sarcasm'), re.IGNORECASE)
    remove_sarcastic = re.compile(re.escape('sarcastic'), re.IGNORECASE)

    for row in csv_file_object:
        if len(row[0:]) == 1:
            temp = row[0:][0]
            temp = remove_hashtags.sub('', temp)
            if len(temp) > 0 and 'http' not in temp and temp[0] != '@' and '\\u' not in temp:
                temp = remove_friendtag.sub('', temp)
                temp = remove_sarcasm.sub('', temp)
                temp = remove_sarcastic.sub('', temp)
                temp = ' '.join(temp.split())  # remove useless space
                if len(temp.split()) > 2:
                    data.append(temp)
                    length.append(len(temp.split()))
    data = list(set(data))
    data = np.array(data)

    return data, length


print('Extracting data')


csv_file_object_pos = csv.reader(open('twitDB_sarcasm.csv', 'rU'), delimiter='\n')
pos_data, length_pos = preprocessing(csv_file_object_pos)


csv_file_object_neg = csv.reader(open('twitDB_regular.csv', 'rU'), delimiter='\n')
neg_data, length_neg = preprocessing(csv_file_object_neg)

print('Number of  sarcastic tweets :', len(pos_data))
print('Average length of sarcastic tweets :', np.mean(length_pos))
print('Number of  non-sarcastic tweets :', len(neg_data))
print('Average length of non-sarcastic tweets :', np.mean(length_neg))

# .npy files...
np.save('posproc', pos_data)
np.save('negproc', neg_data)

In [0]:
#train and test
import nltk
import numpy as np
import scipy as sp
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
import pickle
import heapq

print('Pickling out')
pos_data = np.load('/content/drive/My Drive/Colab Notebooks/posproc.npy')
print(pos_data)
neg_data = np.load('/content/drive/My Drive/Colab Notebooks/negproc.npy')
print(neg_data)
print('Number of  sarcastic tweets :', len(pos_data))
print('Number of  non-sarcastic tweets :', len(neg_data))

print('Training topics')
# topic model
topic()
topic_mod = topic(nbtopic=200, alpha='symmetric')
topic_mod.fit(np.concatenate((pos_data, neg_data)))

print('Feature eng')
# label set
cls_set = ['Non-Sarcastic', 'Sarcastic']
featuresets = []

index = 0
for tweet in pos_data:
    if (np.mod(index, 10000) == 0):
        print("Positive tweet processed: ", index)
    featuresets.append((feature_extract.dialogue_act_features(tweet, topic_mod), cls_set[1]))
    index += 1

index = 0
for tweet in neg_data:
    if np.mod(index, 10000) == 0:
        print("Negative tweet processed: ", index)
    featuresets.append((feature_extract.dialogue_act_features(tweet, topic_mod), cls_set[0]))
    index += 1

featuresets = np.array(featuresets)
targets = (featuresets[0::, 1] == 'Sarcastic').astype(int)

print('Dictionnary vectorizer')
vec = DictVectorizer()
# classifier can only get data in numerical from so convert in vector form.
featurevec = vec.fit_transform(featuresets[0::, 0])

# Saving the dictionnary vectorizer
file_Name = "vecdict.p"
# wb : write binary mode
fileObject = open(file_Name, 'wb')
pickle.dump(vec, fileObject)
fileObject.close()

print('Feature splitting')
# Shuffling
order = shuffle(list(range(len(featuresets))))
# shuffling in same order....
targets = targets[order]
featurevec = featurevec[order, 0::]

# Splitting
size = int(len(featuresets) * .3)  # 30% is used for the test set

trainvec = featurevec[size:, 0::]
train_targets = targets[size:]
testvec = featurevec[:size, 0::]
test_targets = targets[:size]

print('Training')

# Artificial weights
pos_p = (train_targets == 1)
neg_p = (train_targets == 0)
ratio = np.sum(neg_p.astype(float)) / np.sum(pos_p.astype(float))
new_trainvec = trainvec
new_train_targets = train_targets
# THE VERY CORE PART OF WHOLE PROJECT................
for j in range(int(ratio - 1.0)):
    new_trainvec = sp.sparse.vstack([new_trainvec, trainvec[pos_p, 0::]])
    new_train_targets = np.concatenate((new_train_targets, train_targets[pos_p]))

classifier = LinearSVC(C=0.1, penalty='l2', dual=True)
classifier.fit(new_trainvec, new_train_targets)

# Saving the classifier
file_Name = "classif.p"
fileObject = open(file_Name, 'wb')
pickle.dump(classifier, fileObject)
fileObject.close()

# OUR TASK ENDED HERE. OBJECT FILES ARE SAVED. FROM NOW ON JUST PLAYING WITH DATA...........
print('Most important features')

print('grams:')

# coef_ gives weights assigned to features....
coeff = vec.inverse_transform(classifier.coef_[0])[0]
largest = heapq.nlargest(int(100 / 2.0), coeff, key=coeff.get)
smallest = heapq.nsmallest(int(100 / 2.0), coeff, key=coeff.get)
for j in range(int(100 / 2.0)):
    print(largest[j], coeff[largest[j]])
for j in range(int(100 / 2.0)):
    print(smallest[j], coeff[smallest[j]])

print('sentiment:')

print('Positive sentiment', coeff['Positive sentiment'])
print('Positive sentiment 1/2', coeff['Positive sentiment 1/2'])
print('Positive sentiment 2/2', coeff['Positive sentiment 2/2'])
print('Positive sentiment 1/3', coeff['Positive sentiment 1/3'])
print('Positive sentiment 2/3', coeff['Positive sentiment 2/3'])
print('Positive sentiment 3/3', coeff['Positive sentiment 3/3'])
print('Negative sentiment', coeff['Negative sentiment'])
print('Negative sentiment 1/2', coeff['Negative sentiment 1/2'])
print('Negative sentiment 2/2', coeff['Negative sentiment 2/2'])
print('Negative sentiment 1/3', coeff['Negative sentiment 1/3'])
print('Negative sentiment 2/3', coeff['Negative sentiment 2/3'])
print('Negative sentiment 3/3', coeff['Negative sentiment 3/3'])

print('Blob sentiment', coeff['Blob sentiment'])
print('Blob subjectivity', coeff['Blob subjectivity'])
print('Blob sentiment 1/2', coeff['Blob sentiment 1/2'])
print('Blob sentiment 2/2', coeff['Blob sentiment 2/2'])
print('Blob subjectivity 1/2', coeff['Blob subjectivity 1/2'])
print('Blob subjectivity 2/2', coeff['Blob subjectivity 2/2'])
print('Blob sentiment 1/3', coeff['Blob sentiment 1/3'])
print('Blob sentiment 2/3', coeff['Blob sentiment 2/3'])
print('Blob sentiment 3/3', coeff['Blob sentiment 3/3'])
print('Blob subjectivity 1/3', coeff['Blob subjectivity 1/3'])
print('Blob subjectivity 2/3', coeff['Blob subjectivity 2/3'])
print('Blob subjectivity 3/3', coeff['Blob subjectivity 3/3'])

print('topics:')

topics_tag = []
topics_coeff = []
topics_num = []
for j in range(200):
    topics_tag.append('Topic :' + str(j))
    topics_coeff.append(coeff[topics_tag[j]])
    topics_num.append(j)
topics_tag = np.array(topics_tag)
topics_num = np.array(topics_num)
topics_coeff = np.array(topics_coeff)

# Returns the indices that would sort the array.
topics_num = topics_num[topics_coeff.argsort()]
topics_tag = topics_tag[topics_coeff.argsort()]
topics_coeff = topics_coeff[topics_coeff.argsort()]
for j in range(10):
    print(topics_coeff[j], topic_mod.get_topic(topics_num[j]))
for j in range(190, 200):
    print(topics_coeff[j], topic_mod.get_topic(topics_num[j]))

print('Validating')

output = classifier.predict(testvec)
print(classification_report(test_targets, output, target_names=cls_set))

# BASIC TEST
basic_test = ["This is just a long sentence, to make sure that it's not how long the sentence is that matters the most",
              'I just love when you make me feel like shit', 'Life is odd', 'Just got back to the US !',
              "Isn'it great when your girlfriend dumps you ?", "I love my job !", 'I love my son !', "I love my life !!"]
feature_basictest = []
for tweet in basic_test:
    # get feature vector for each tweet
    feature_basictest.append(feature_extract.dialogue_act_features(tweet, topic_mod))
feature_basictest = np.array(feature_basictest)
# classifier can only get data in numerical from so convert in vector form.
feature_basictestvec = vec.transform(feature_basictest)

print(basic_test)
print(classifier.predict(feature_basictestvec))
print(classifier.decision_function(feature_basictestvec))

Pickling out
[b'My family is so open and loving.'
 b"What who did to who? Others? Let America decide what to do? That's helpful advice!"
 b'and texts dont go' ... b'I love having so many friends'
 b'WoW what a great night.' b'now listening to: acdc-back in black\\n']
[b'Yo $20 to gets me and takes me to mcdonalds'
 b'A nigga will fuck anything ugh'
 b"I hate when people try to tell me about me. You don't know me." ...
 b'Anyone wanna go kayaking with me tomorrow'
 b'Finals MVP Green or Leonard'
 b'Success Is Not Final Failure Is Not Fatal: It Is The Courage To Continue That Counts.']
Number of  sarcastic tweets : 25273
Number of  non-sarcastic tweets : 117825
Training topics


NameError: ignored

In [0]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Note: To use the 'upload' functionality of this file, you must:
pip install twine

import io
import os
import sys
from shutil import rmtree

from setuptools import find_packages, setup, Command

# Package meta-data.
NAME = 'deeptranslit'
DESCRIPTION = 'Transliteration with Deep Learning'
URL = 'https://github.com/bedapudi6788/deeptranslit'
EMAIL = 'praneethbedapudi@gmail.com'
AUTHOR = 'BEDAPUDI PRANEETH'
REQUIRES_PYTHON = '>=3.6.0'
VERSION = '1.0.1'

# What packages are required for this module to be executed?
REQUIRED = [
    'txt2txt'
]

# What packages are optional?
EXTRAS = {
    # 'fancy feature': ['django'],
}

# The rest you shouldn't have to touch too much :)
# ------------------------------------------------
# Except, perhaps the License and Trove Classifiers!
# If you do change the License, remember to change the Trove Classifier for that!


here = os.path.abspath(os.path.dirname('__file__'))

# Import the README and use it as the long-description.
# Note: this will only work if 'README.md' is present in your MANIFEST.in file!
try:
    with io.open(os.path.join(here, 'README.md'), encoding='utf-8') as f:
        long_description = '\n' + f.read()
except FileNotFoundError:
    long_description = DESCRIPTION

# Load the package's __version__.py module as a dictionary.
about = {}
if not VERSION:
    with open(os.path.join(here, NAME, '__version__.py')) as f:
        exec(f.read(), about)
else:
    about['__version__'] = VERSION


class UploadCommand(Command):
    """Support setup.py upload."""

    description = 'Build and publish the package.'
    user_options = []

    @staticmethod
    def status(s):
        """Prints things in bold."""
        print('\033[1m{0}\033[0m'.format(s))

    def initialize_options(self):
        pass

    def finalize_options(self):
        pass

    def run(self):
        try:
            self.status('Removing previous builds…')
            rmtree(os.path.join(here, 'dist'))
        except OSError:
            pass

        self.status('Building Source and Wheel (universal) distribution…')
        os.system('{0} setup.py sdist bdist_wheel --universal'.format(sys.executable))

        self.status('Uploading the package to PyPI via Twine…')
        os.system('twine upload dist/*')

        self.status('Pushing git tags…')
        os.system('git tag v{0}'.format(about['__version__']))
        os.system('git push --tags')
        
        sys.exit()


# Where the magic happens:
setup(
    name=NAME,
    version=about['__version__'],
    description=DESCRIPTION,
    long_description=long_description,
    long_description_content_type='text/markdown',
    author=AUTHOR,
    author_email=EMAIL,
    python_requires=REQUIRES_PYTHON,
    url=URL,
    packages=find_packages(exclude=('tests',)),
    # If your package is a single module, use this instead of 'packages':
    # py_modules=['mypackage'],

    # entry_points={
    #     'console_scripts': ['mycli=mymodule:cli'],
    # },
    install_requires=REQUIRED,
    extras_require=EXTRAS,
    include_package_data=True,
    license='GPLv3',
    classifiers=[
        # Trove classifiers
        # Full list: https://pypi.python.org/pypi?%3Aaction=list_classifiers
        'License :: OSI Approved :: GNU General Public License v3 (GPLv3)',
        'Programming Language :: Python',
        'Programming Language :: Python :: 3',
        'Programming Language :: Python :: 3.6',
        'Programming Language :: Python :: Implementation :: CPython',
        'Programming Language :: Python :: Implementation :: PyPy'
    ],
    #setup.py publish support.
    cmdclass={
        'upload': UploadCommand,
    },
)   

In [0]:
import os
import re
import pickle
import string
import pydload
import logging
import itertools

from txt2txt import infer, build_model

kenlm_available = True

try:
    import kenlm
except:
    logging.warn('KenLm not installed. Simple scoring will be used.')
    kenlm_available = False

model_links = {
            'hi': {
                    'checkpoint': 'https://github.com/bedapudi6788/DeepTranslit/releases/download/v0.5/en_hi_checkpoint',
                    'params': 'https://github.com/bedapudi6788/DeepTranslit/releases/download/v0.5/en_hi_params',
                    'words': 'https://github.com/bedapudi6788/DeepTranslit/releases/download/v0.5/hi_words',
                    'lm': 'https://github.com/bedapudi6788/DeepTranslit/releases/download/v0.5/hi_lm.bin'
                },
            }

lang_code_mapping = {
            'hindi': 'hi',
        }

class DeepTranslit():
    params = None
    model = None
    words = None
    lm = None
    rank = 'auto'

    def __init__(self, lang_code, rank='auto'):
        """
        Initialize deeptranslit
        Parameters:
        lang_code (str): Name or code of the language. (Currently supported: hindi/hi)
        rank (str): Mode of ranking. In default mode ('auto') kenlm will be used if available. (simple|kenlm|auto are the supported options)
        """

        if lang_code in lang_code_mapping:
            lang_code = lang_code_mapping[lang_code]
        
        if lang_code not in model_links:
            print("DeepTranslit doesn't support '" + lang_code + "' yet.")
            print("Please raise a issue at https://github.com/bedapudi6788/deeptranslit to add this language into future checklist.")
            return None
        
        # loading the model
        home = os.path.expanduser("~")
        lang_path = os.path.join(home, '.DeepTranslit_' + lang_code)
        checkpoint_path = os.path.join(lang_path, 'checkpoint')
        params_path = os.path.join(lang_path, 'params')
        words_path = os.path.join(lang_path, 'words')
        lm_path = os.path.join(lang_path, 'lm')
        
        if not os.path.exists(lang_path):
            os.mkdir(lang_path)

        if not os.path.exists(checkpoint_path):
            print('Downloading checkpoint', model_links[lang_code]['checkpoint'], 'to', checkpoint_path)
            pydload.dload(url=model_links[lang_code]['checkpoint'], save_to_path=checkpoint_path, max_time=None)

        if not os.path.exists(params_path):
            print('Downloading model params', model_links[lang_code]['params'], 'to', params_path)
            pydload.dload(url=model_links[lang_code]['params'], save_to_path=params_path, max_time=None)
        
        if not os.path.exists(words_path):
            print('Downloading words', model_links[lang_code]['words'], 'to', words_path)
            pydload.dload(url=model_links[lang_code]['words'], save_to_path=words_path, max_time=None)

        if not os.path.exists(lm_path):
            print('Downloading lm', model_links[lang_code]['lm'], 'to', lm_path)
            pydload.dload(url=model_links[lang_code]['lm'], save_to_path=lm_path, max_time=None)
        
        DeepTranslit.model, DeepTranslit.params = build_model(params_path=params_path, enc_lstm_units=64, use_gru=True, display_summary=False)
        DeepTranslit.model.load_weights(checkpoint_path)

        DeepTranslit.words = pickle.load(open(words_path, 'rb'))

        if kenlm_available and rank in {'auto', 'kenlm'}:
            logging.warn('Loading KenLM.')
            DeepTranslit.lm = kenlm.Model(lm_path)
            DeepTranslit.rank = rank

    def transliterate(self, sent, top=3):
        """
        Transliterate an input sentence while preserving punctuation at word or sentence endings.
        Parameters:
        sent (str): Sentence to be transliterated.
        top (int): top-n results to be returned. if 0 or None, all results will be returned.
        Returns:
        list: returns list of tuples of size 2 with first element of each tuple being the transliterated sentence and second element being the "score"
        """
        rank = DeepTranslit.rank
        words = sent.strip().split()
        puncs = []
        for i, word in enumerate(words):
            words[i] = re.sub('[' + string.punctuation + ']', '', word.lower())
            if not words[i]:
                continue

            punc = None
            if word[-1] in string.punctuation:
                punc = word[-1]
            
            puncs.append(punc)

        words = [w for w in words if w]
        
        np_words = []

        for i, word in enumerate(words):
            if [c for c in word if c not in DeepTranslit.params['input_encoding']]:
                np_words.append((i - len(np_words), word))
                words[i] = None
            
        words = [w for w in words if w]        

        preds = infer(words, DeepTranslit.model, DeepTranslit.params)

        for posi, np_word in np_words:
            preds = preds[:posi] + [[{'sequence': np_word, 'prob': 1}]] + preds[posi:]

        resp = []

        preds = list(itertools.product(*preds))

        for pred in preds:
            words = [w['sequence'] for w in pred]
            for i, word in enumerate(words):
                if puncs[i]:
                    word = word + puncs[i]
                words[i] = word

            probs = [w['prob'] for w in pred]

            sent = ' '.join(words)
            resp.append((sent, probs))
        
        if rank == 'auto':
            if kenlm_available:
                rank = 'kenlm'
            else:
                rank = 'simple'

        if rank == 'simple':
            for i, (sent, probs) in enumerate(resp):
                words = sent.split()
                score = sum([1 for word in words if word in DeepTranslit.words])
                resp[i] = (sent, score)
            
            resp = sorted(resp, key=lambda x: x[1], reverse=True)
        
        elif rank == 'kenlm':
            if not kenlm_available:
                logging.error("KenLm not available")
                return resp

            for i, (sent, probs) in enumerate(resp):
                score = DeepTranslit.lm.score(sent)
                resp[i] = (sent, score)

            resp = sorted(resp, key=lambda x: x[1], reverse=True) 
        

        if top:
            resp = resp[:top]

        return resp

In [0]:
#transliterator = DeepTranslit('hindi')
# Currently hindi (hi) is the only supported language.

#transliterator.transliterate('mai mbbs complete karliya')